# 3.3 데이터 전처리
- 타깃 변수에 영향을 끼치는 변수들을 생성 

- 데이콘 경진대회 1등 솔루션 ch03

In [38]:
import pandas as pd

In [39]:
# 데이터 로드
train = pd.read_csv("ch03/train.csv")
test = pd.read_csv("ch03/test.csv")
bts=pd.read_csv("ch03/bus_bts.csv")
jeju_life = pd.read_csv("ch03/jeju_financial_life_data.csv")
weather = pd.read_csv("ch03/weather.csv", encoding='cp949')
rain = pd.read_csv("ch03/rain.csv", encoding='utf-8')

In [40]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 내부에 결과를 출력하도록 설정
%matplotlib inline
# 시각화 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False) 
# 분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings
warnings.filterwarnings('ignore') 

* 학습 데이터와 테스트 데이터가 동일한 변수를 가져야한다. 학습 데이터와 테스트 데이터를 구분할 수 있는 cue변수를 추가한 후 병합하여 변수 생성 

In [41]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성 
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터를 통합 
df = pd.concat([train, test], axis=0)
df.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,cue
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


## 3.3.1 내부 데이터를 통한 변수 생성

* 요일을 나타내는 변수 (weekday)

In [42]:
# datetime 변수형으로 변환 
df['date'] = pd.to_datetime(df['date'])

# 요일 추출(0- 월요일, 6- 일요일)
df['weekday'] = df['date'].dt.weekday

df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


* 요일별 평균 탑승객 수

In [43]:
# 요일별 평균 탑승객 수를 구하는 함수 
def week_mean():
    # 전체 데이터에서 train 데이터에 해당하는 행 추출 
    train_data = df.query('cue==0').reset_index(drop=True)

    # 일괄적으로 1의 값을 가지는 weekdaymean 변수 생성 
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출 
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용해 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 탑승객 수로 변경 
    df.iloc[index0, -1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1, -1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2, -1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3, -1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4, -1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5, -1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6, -1] = train_data.query('weekday==6')['18~20_ride'].mean()

    return df

# 함수를 실행하여 변수 생성 
df = week_mean()

In [44]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


* 버스 종류별 평균  탑승객 수(in_out_mean)

In [45]:
# 버스 종류별 평균 탑승객 수를 구하는 함수 
def inout_mean():
    # 전체 데이터에서 train 데이터에 해당하는 행 추출 
    train_data = df.query('cue==0').reset_index(drop=True)

    # 일괄적으로 1의 값을 가지는 in_out_mena 변수 생성 
    df['in_out_mean'] = 1

    # 버스 종류 인덱스 추출 
    in_index = df.query('in_out == "시내"').index
    out_index = df.query('in_out== "시외"').index

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는 평균 탑승 승객 수로 변경 
    df.iloc[in_index, -1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index, -1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()

    return df

# 함수로 실행하여 변수 설정
df = inout_mean()


In [46]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


* 일별 오전 시간대의 총 탑승객수 
    - 오전 시간대의 탑승객 수는 해당 하는 날짜의 퇴근 시간 탑승객 수의 영향ㅇ르 줄 수 있다 
    - 출,퇴근 탑승객수 비슷항 양상을 보인다 

In [47]:
# 날짜별 오전 시간에 탑승한 총 탑승객 수
f =df.groupby('date')['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride'].sum().reset_index()

# 변수명 변경 
f.columns= ['date', '6~7_ride_sum', '7~8_ride_sum','8~9_ride_sum','9~10_ride_sum', '10~11_ride_sum']

# 기존 데이터프레임에 새로운 변수를 병합 
df = pd.merge(df, f, how='left', on='date')

In [48]:
df[['date', '6~7_ride_sum', '7~8_ride_sum','8~9_ride_sum','9~10_ride_sum', '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


### 도메인 조사를 통한 변수 

* 버스 배차 간격(bus_interval)
    - 일정 시간 간격으로 버스를 배치할 때 이전 버스와의 시간 간격 
    - 배차 간격이 짧으면 수요가 많은 버스임을 유출 할 수 있다. 
<br>
* bus_interval 변수 구하기 
    - 버스 번호는 비식별 처리로 버스 배차 간격을 구분하기 어려움 
    

In [49]:
# 탑승날짜, 탑승 정류장 코드 탑승 노선 번호 순으로 시간에 따라 정렬 

# 탑승 날짜를 datetime형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date','geton_station_code','bus_route_id',
                        'geton_time2'],ascending=True).reset_index()

In [50]:
# 상태 진행률을 확인할 수 있은 tqdm 라이브러리 사용 
from tqdm import notebook 

# 시차를 넣기 위한 빈 리스트 생성
interval_lst = []
for i in notebook.tqdm(range(0, len(f)-1)):
    # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date)&
         (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &
         (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
         # 탑승 날짜의 차를 interval_lst에 넣기 
         interval_lst.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)
    else:
        # 같지 않다면 0을 interval_lst에 넣기 
        interval_lst.append(0)

interval_lst.insert(0, '0')

  0%|          | 0/2409413 [00:00<?, ?it/s]

In [51]:
# 값의 형태 통일 시키기 

# interval_lst를 'interval'변수로 생성 
f['interval'] = interval_lst 
# 'interval' 변수의 값을 문자형으로 변환 후 '시:분:초'의 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('','00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [52]:
# 값의 형태 통일 시키기 

# interval_lst 를 interval 변수로 생성 
f['interval'] = interval_lst 

# interval 변수의 값을 문자형으로 변환 후 시:분:초 형태로 변환 
f['interval'] = f['interval'].astype(str).str[7:].replace('', '00:00:00')
f['bus_route_id'] =f['bus_route_id'].astype(object)

* interval 변수의 값을 '분'으로 변환 하기

In [26]:
# 분으로 변환하는 함수 
def get_min(time_str):
    h,m,s = time_str.split(':')
    return(int(h)*3600 + int(m)*60 + int(s))/ 60

# interval 함수의 모든 값에 대해 함수 적용하여 나온 값을 time 리스트에 넣기 
time = []
for i in f['interval']:
    time.append(get_min(i))

f['time'] = time
f['time'] = f['time'].astype(int)
# 간격이 3분 보다 작은 것은 제외 
f = f[f['time'] > 3 ]
# 간격이 3시간 보다 큰 것은 제외 
f = f[f['time'] < 180]

In [28]:
# 버스 노선별 버스 간격의 평균 bus_interval 변수 생성 

data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

data.head()

# 변수 생성에 많은 시간이 소요 되므로 CSV 파일 저장 
data.to_csv('bus_interval.csv', index=False)

* 버스 노선 번호에 따른 배차 간격(분)

In [53]:
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv('bus_interval.csv')

# 기존 데이터 프레임에 새로운 변수 병합 
df = pd.merge(df, bus_interval, how='left', on='bus_route_id')

# 병합 중 생기는 NA값 처리 
# 배차 간격이 짧을 수록 수요가 많은 버스임을 의미 함으로 결측치를 0으로 두는 것은 변수 생성 목적과 다름 
#  결측치를 9999 변경 
df['bus_interval'] = df['bus_interval'].fillna(9999)

In [54]:
df[['bus_interval']].head()

,bus_interval
0,34.0
1,34.0
2,34.0
3,34.0
4,34.0


* 수요가 많을 것으로 예상되는 정류장(school, transfer)
    - school 변수 생성


In [57]:
# 정류장명에 '고등학교' 포함되어 있는 행 뽑기
highschool_station = df[df['station_name'].str.contains('고등학교')]
# 고등학교가 포함되어 있는 정류장명을 중복 없이 리스트에 저장 
highschool = list(highschool_station['station_name'].unique())
# 정류장명에 '대학교' 포함되어 있는 행 뽑기
university_station = df[df['station_name'].str.contains('대학교')]
# 대학교가 포함되어 있는 정류장명을 중복 없이 리스트에 저장 
university = list(university_station['station_name'].unique())


# 고등학교 와 대학교 근처 정류장 구분 함수 
def schoolstation(x):
    # highschool 리스트에 정류장명이 포함되어 있다면 1을 반환 
    if x in highschool:
        return 1
    # university 리스트에 정류장명이 포함되어 있다면 
    elif x in university:
        return 1
    else:
        return 0

# 모든 정류장명에 대해 적용 
df['school'] = df['station_name'].apply(schoolstation)

In [58]:
df[['school']].head()

,school
0,0
1,0
2,0
3,0
4,0


- 환승 정류장

In [59]:
# 환승
transfer_station = df[df['station_name'].str.contains('환승')]
transfers = list(transfer_station['station_name'].unique())

# 공항 
airport_station = df[df['station_name'].str.contains('공항')]
airport = list(airport_station['station_name'].unique())

# 터미널 
terminal_station = df[df['station_name'].str.contains('터미널')]
terminal = list(terminal_station['station_name'].unique())


def transfer_station(x):
    if x in transfers:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

# 모든 정류장명에 대해 적용 
df['transfers'] = df['station_name'].apply(transfer_station)

In [60]:
df[['transfers']].head()

,transfers
0,0
1,0
2,0
3,1
4,0


- 연휴(holiday)
    - 연휴 기간에는 학교와 회사에 가지 않으므로 퇴근 시간에 평소와 다른 수요 패턴 

In [61]:
def holiday_function(x):
    # 공휴일 이라면 1을 반환 
    if x in ['2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09']:
        return 1
    else:
        return 0
    
df['holiday'] = df['date'].apply(holiday_function)

In [62]:
df[['holiday']].head()

,holiday
0,0
1,0
2,0
3,0
4,0


### 시간대를 활용한 변수
- 오전 시간대를 활용해 변수를 만드는 방법 
    - 18시 ~ 20시 까지 2시간엔 해당하는 승차 인원 예측을 우 ㅣ해 2시간 간격으로 승하차 시간대를 생성

In [63]:
# 오전 시간대에 2시간 동안 승차한 인원수의 합계 
df['68a'] = df['6~7_ride'] + df['7~8_ride']
df['810a'] = df['8~9_ride'] + df['9~10_ride']
df['1012a'] = df['10~11_ride'] + df['11~12_ride']

# 오전 시간대에 2시간 동안 하차한 인원수의 합계 
df['68b'] = df['6~7_takeoff'] + df['7~8_takeoff']
df['810b'] = df['8~9_takeoff'] + df['9~10_takeoff']
df['1012b'] = df['10~11_takeoff'] + df['11~12_takeoff']

* 오전 시간의 승객 수 (ride_sume, takeoff_sum)

In [65]:
# 오전시간에 승차한 승객 수의 합계 
df['ride_sum'] = df[['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
                      '10~11_ride', '11~12_ride']].sum(axis=1)

# 오전시간에 하차한 승객 수의 합계 
df['takeoff_sum']= df[['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff',
                      '10~11_takeoff', '11~12_takeoff']].sum(axis=1)
            

df[['ride_sum', 'takeoff_sum']].head()

,ride_sum,takeoff_sum
0,16.0,0.0
1,22.0,0.0
2,4.0,0.0
3,79.0,0.0
4,0.0,1.0


* 노선별 오전 시간대의 총 평균 탑승객수 

In [68]:
# bus_route_id 와 station_code를 결합해 route_station 생성 
df['route_station'] = df['bus_route_id'].astype(str) + ','+ df['station_code'].astype(str)

# 버스 노선별 오전 시간의 총 탑승객 수와  평균 탑승객 수 구하기 
data = df.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()

# 변수명을 1012a_sum, 1012a_mean
data.rename(columns={'mean':'1012a_mean', 'sum':'1012a_sum'}, inplace=True)

# 기존 데이터 프레임에 새로운 변수를 병합 
df = pd.merge(df, data, how='left', on='route_station')

### bus_bts 데이터를 활용한 변수 
- 카테고리별 승객 수의 합과 비율 

In [76]:
# 카테고리별 승객 수의 합 
def category_people():
    # df와 병합을 위해 bus_route_id의 타입을 str 로 통일 
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    # bus_route_id별 user_category 별 승객 수의 합 
    f = bts.groupby(['bus_route_id', 'user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    # bus_route_id별 user_category 변수의 승객 수의 피벗 테이블 형태로 변환 
    g = pd.pivot_table(f, values='승객수', index='bus_route_id', columns='user_category', fill_value=0).reset_index()
    g.columns=['bus_route_id', 'adult', 'kids', 'teen', 'elder', 'jang', 'jang2', 'ugong', 'ugong2'] 
    g = g[['bus_route_id', 'adult', 'kids', 'teen', 'elder']]


    # 기존 데이터프레임에 새로운 변수를 병합 
    merge_data = pd.merge(df, g, how='left', on='bus_route_id')
    return merge_data 

# bus_route_id 의 타입을 str로 통일시킨뒤, 함수를 실행하여 변수 생성 
df['bus_route_id'] = df['bus_route_id'].astype(str) 
df = category_people()  

In [78]:
df[['bus_route_id', 'adult', 'kids', 'teen', 'elder']].head()

,bus_route_id,adult,kids,teen,elder
0,4270000,8158.0,72.0,38.0,0.0
1,4270000,8158.0,72.0,38.0,0.0
2,4270000,8158.0,72.0,38.0,0.0
3,4270000,8158.0,72.0,38.0,0.0
4,4270000,8158.0,72.0,38.0,0.0


In [81]:
# 카테고리별 승객 수의 비율 
def category_people_ratio():

    # bus_route_id별 user_category의 전체합 
    f_all = bts.groupby('bus_route_id')['user_count'].agg([('전체', 'sum')]).reset_index()

    # bust_route_id별 category별 승객 수의 합 
    f_sum = bts.groupby(['bus_route_id', 'user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    # 승객 수의 합을 전체 합으로 나눠 비율을 구함 
    data = pd.merge(f_all, f_sum, on='bus_route_id')
    data['비율'] = data['승객수'] / data['전체']

    # 피벗 테이블을 통해 bus_route_id별 user_cateogry 변수의 승객 비율 형태로 변환 
    data = pd.pivot_table(data, values='비율', index='bus_route_id',columns='user_category', fill_value=0).reset_index()
    data.columns = ['bus_route_id', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop', 'jang_prop', 'jang2_prop', 'ugong_prop', 'ugong2_prop']
    f = data[['bus_route_id', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop']]

    merge_data =pd.merge(df, f, how='left', on='bus_route_id')
    return merge_data


df= category_people_ratio()

In [82]:
df[['adult_prop', 'kids_prop', 'teen_prop', 'elder_prop']].head()

,adult_prop,kids_prop,teen_prop,elder_prop
0,0.986696,0.008708,0.004596,0.0
1,0.986696,0.008708,0.004596,0.0
2,0.986696,0.008708,0.004596,0.0
3,0.986696,0.008708,0.004596,0.0
4,0.986696,0.008708,0.004596,0.0


* 결측치 처리, 대체 
    - 결측값이 존재하는 변수들은 모두 연속형 변수의 평균값으로 대체 가능

In [83]:
# 연속형 변수의 결측치를 각 변수의 평균값으로 처리할 수 있도록 SimpleImputer를 사용 
from sklearn.impute import SimpleImputer 

# SimpleImputer를 사용할 변수 추출 
na_col = ['adult', 'kids', 'teen', 'elder', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop']

# 평균값으로 결측치 처리를 위해 strategy 옵션을 mean으로 설정 
imputer = SimpleImputer(strategy= 'mean')

# 각 변수에 대해 평균값을 구하고 결측치 대치를 이해 fit_transfrom 
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns=na_col)

### 좌표를 활용한 변수
- 인구가 밀집된 지역(서귀포, 제주)일 수로고 탑승객 수가 많은 것이다 라는 예측

In [87]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) # 제주시의 위도 경도
coords_seoquiposi = (33.259429, 126.558217) # 서귀포시의 위도 경도

# 제주시와의 거리
df['dis_jejusi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]
# 서귀포시와의 거리
df['dis_seoquiposi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_seoquiposi).km for i in range(len(df))]

In [88]:
df[['dis_jejusi', 'dis_seoquiposi']].head()

,dis_jejusi,dis_seoquiposi
0,2.954920,26.256744
1,3.720275,26.403025
2,5.036124,25.893305
3,2.864166,27.997494
4,29.040353,13.574693


* 탑승하는 승객의 수가 많은 버스정류장과의 거리
    - 사람이 많이 타는 버스 정류장 근처는 유동인구가 많은 지역 

In [89]:
# 승차 인원 상위 10개의 버스정류장 좌표
coords_jejusicheong1 = (33.49892, 126.53035) # 제주시청(광양 방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) # 제주국제공항(구제주 방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) # 한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) # 제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) # 동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) # 제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) # 중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양 방면)의 위도 경도

# 승차 인원 상위 10개의 버스정류장 좌표와 각 정류장의 지구 표면에서 두 점 사이의 거리를 구함
df['dis_jejusicheong1'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(df))]
df['dis_jejuairport'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(df))]
df['dis_hallahosp'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(df))]
df['dis_rotary'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_rotary).km for i in range(len(df))]
df['dis_jejucenterhigh'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(df))]
df['dis_jejumarket'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(df))]
df['dis_jejusclass'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(df))]
df['dis_centerroad'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_centerroad).km for i in range(len(df))]
df['dis_fiveway'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_fiveway).km for i in range(len(df))]
df['dis_law'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_law).km for i in range(len(df))]